In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.metrics import mean_squared_error

import xgboost as xgb

/home/vladimir/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_csv("X_train_processed.csv", sep=',')

In [17]:
df_train.shape

(270000, 12)

In [3]:
class ModelsEstimator:
    n_models = 20
    
    def __init__(self):
        self.models = [xgb.XGBRegressor(max_depth=6, n_estimators=100)
                       for i in range(self.n_models)]
        
    def fit(self, x, y):
        sample_size = len(x)
        sub_sample_size = sample_size # int(2.0 / 3.0 * sample_size)
        
        for model_i, model in enumerate(self.models):
            index = np.random.choice(sample_size, size=sub_sample_size, replace=True)
            X_train, Y_train = x[index], y[index]
            model.fit(X_train, Y_train)
            # print '\r', model_i,
        
        # print "\rFitted!"
        
    def predict_per_model(self, x):
        return [model.predict(x) for model in self.models]
    
    def predict(self, x):
        return np.mean(self.predict_per_model(x), axis=0)

In [4]:
estimator = ModelsEstimator()

In [5]:
estimator.fit(df_train.values[:, 1:-1], df_train.values[:, -1])

In [6]:
X_test = pd.read_csv("../X_private.csv", sep=',')
X_test.head()

,id,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9
0,0,0.220902,1.704661,3.145539,2.069166,3.248110,1.655190,0.336612,1.794711,0.719335,2.405603
1,1,4.970893,2.728854,1.974131,1.230789,1.915259,3.573415,2.897298,4.279815,0.255498,0.682067
2,2,4.261701,0.055729,1.623474,2.224815,4.037181,2.625306,2.679721,4.849078,3.586963,3.269388
3,3,4.109848,2.355999,2.030721,0.741282,3.873136,0.900767,0.493190,2.479181,0.666730,2.686895
4,4,4.268486,4.547079,3.115409,4.988304,0.899760,4.486683,0.915195,1.881219,4.733739,4.261041


In [7]:
Y_pred = estimator.predict(X_test.values[:, 1:])

In [14]:
submission = [(i, y) for i, y in zip(X_test["id"], Y_pred)]
submission = pd.DataFrame(submission, columns=["id", "y"])

In [9]:
submission.head()

,id,y
0,0,-9.810095e+07
1,1,-4.928236e+06
2,2,-1.550364e+07
3,3,5.510236e+07
4,4,1.007039e+10


In [15]:
submission.to_csv("../submission_final.csv", sep=',', index=False)